In [1]:
import glob
import csv
import tqdm
import copy
import numpy as np
import pandas as pd
import catboost
import math

# Load track features into dataframe

In [2]:
track_features_dir = "/media/data2/Data/wsdm2019/data/track_features/"
track_feature_files = sorted(glob.glob(track_features_dir+"*.csv"))
track_feature_dir_0 = track_feature_files[0]
track_feature_0 = pd.read_csv(track_feature_dir_0)

track_feature_dir_1 = track_feature_files[1]
track_feature_1 = pd.read_csv(track_feature_dir_1)

track_feature_all = track_feature_0.append(track_feature_1)

# Load session features into dataframe

In [3]:
train_set_dir = "/media/data2/Data/wsdm2019/data/training_set/"
train_files = sorted(glob.glob(train_set_dir+"*.csv"))

catboost_files = train_files[500:]

temp_catboost_train_file_size = 5
temp_catboost_valid_file_size = 1

catboost_train_files = catboost_files[:temp_catboost_train_file_size]
catboost_valid_files = catboost_files[temp_catboost_train_file_size:temp_catboost_valid_file_size + temp_catboost_train_file_size]

In [4]:
train_df_list = []
valid_df_list = []

for f in catboost_train_files:
    train_df_list.append(pd.read_csv(f))    

for f in catboost_valid_files:
    valid_df_list.append(pd.read_csv(f))
    
catboost_train_raw = pd.concat(train_df_list, ignore_index=True)
catboost_valid_raw = pd.concat(valid_df_list, ignore_index=True)

In [5]:
catboost_train_raw.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,38_00012a3f-dff9-4560-b18b-103cfecf8a43,1,11,t_e2dda38d-f826-481b-b49a-a1d9051f5c6d,False,True,True,False,0,0,...,0,1,0,False,1,2018-08-22,True,editorial_playlist,remote,trackdone
1,38_00012a3f-dff9-4560-b18b-103cfecf8a43,2,11,t_f441b003-ce0e-4723-b534-4a3f1286f33e,False,False,False,True,0,1,...,0,0,0,False,1,2018-08-22,True,editorial_playlist,trackdone,trackdone
2,38_00012a3f-dff9-4560-b18b-103cfecf8a43,3,11,t_5c99f04b-47c6-4307-bb21-75b068b5d078,False,False,True,False,0,0,...,1,0,0,False,1,2018-08-22,True,editorial_playlist,trackdone,fwdbtn
3,38_00012a3f-dff9-4560-b18b-103cfecf8a43,4,11,t_47cfab15-2b20-4dbb-91c1-db3b8cb70036,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,fwdbtn,trackdone
4,38_00012a3f-dff9-4560-b18b-103cfecf8a43,5,11,t_ab40acd6-2781-41e2-8c38-61cf49e02999,False,True,True,False,0,0,...,1,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,fwdbtn


In [6]:
catboost_train_raw.shape

(15502680, 21)

In [7]:
catboost_valid_raw.shape

(3092605, 21)

In [8]:
catboost_train_raw_group = catboost_train_raw.groupby('session_id')
ngroups = catboost_train_raw_group.ngroups

In [9]:
print(ngroups)

940033


In [ ]:
# for name, group in df_group:
    

# catboost_train_raw_group_history = catboost_train_raw.groupby('session_id').apply(lambda x: x.iloc[:math.floor(x['session_position'].size / 2)])
# catboost_train_raw_group_predict = catboost_train_raw.groupby('session_id').apply(lambda x: x.iloc[math.floor(x['session_position'].size / 2):])

# catboost_train_raw_group_history.get_group('38_00012a3f-dff9-4560-b18b-103cfecf8a43')

# catboost_train_raw_group_predict.get_group('38_00012a3f-dff9-4560-b18b-103cfecf8a43')

In [10]:
test_group = catboost_train_raw_group.get_group('38_00012a3f-dff9-4560-b18b-103cfecf8a43')

In [11]:
test_group

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,38_00012a3f-dff9-4560-b18b-103cfecf8a43,1,11,t_e2dda38d-f826-481b-b49a-a1d9051f5c6d,False,True,True,False,0,0,...,0,1,0,False,1,2018-08-22,True,editorial_playlist,remote,trackdone
1,38_00012a3f-dff9-4560-b18b-103cfecf8a43,2,11,t_f441b003-ce0e-4723-b534-4a3f1286f33e,False,False,False,True,0,1,...,0,0,0,False,1,2018-08-22,True,editorial_playlist,trackdone,trackdone
2,38_00012a3f-dff9-4560-b18b-103cfecf8a43,3,11,t_5c99f04b-47c6-4307-bb21-75b068b5d078,False,False,True,False,0,0,...,1,0,0,False,1,2018-08-22,True,editorial_playlist,trackdone,fwdbtn
3,38_00012a3f-dff9-4560-b18b-103cfecf8a43,4,11,t_47cfab15-2b20-4dbb-91c1-db3b8cb70036,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,fwdbtn,trackdone
4,38_00012a3f-dff9-4560-b18b-103cfecf8a43,5,11,t_ab40acd6-2781-41e2-8c38-61cf49e02999,False,True,True,False,0,0,...,1,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,fwdbtn
5,38_00012a3f-dff9-4560-b18b-103cfecf8a43,6,11,t_5478b5f2-f2bb-4ebb-808b-9c14ee50950c,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,fwdbtn,trackdone
6,38_00012a3f-dff9-4560-b18b-103cfecf8a43,7,11,t_8db3f89c-510b-4e30-87bc-887502bd9030,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
7,38_00012a3f-dff9-4560-b18b-103cfecf8a43,8,11,t_696ab8a4-321b-46fc-ac04-36fc07d37a30,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
8,38_00012a3f-dff9-4560-b18b-103cfecf8a43,9,11,t_ceb4e1b4-1cf6-432a-b199-107a0aef045b,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
9,38_00012a3f-dff9-4560-b18b-103cfecf8a43,10,11,t_b918f474-f39d-4bbc-89f9-463001ca1510,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone


In [14]:
test_group_history = test_group.iloc[:math.floor(test_group['session_position'].size / 2)]
test_group_predict = test_group.iloc[math.floor(test_group['session_position'].size / 2):]

In [15]:
test_group_history

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,38_00012a3f-dff9-4560-b18b-103cfecf8a43,1,11,t_e2dda38d-f826-481b-b49a-a1d9051f5c6d,False,True,True,False,0,0,...,0,1,0,False,1,2018-08-22,True,editorial_playlist,remote,trackdone
1,38_00012a3f-dff9-4560-b18b-103cfecf8a43,2,11,t_f441b003-ce0e-4723-b534-4a3f1286f33e,False,False,False,True,0,1,...,0,0,0,False,1,2018-08-22,True,editorial_playlist,trackdone,trackdone
2,38_00012a3f-dff9-4560-b18b-103cfecf8a43,3,11,t_5c99f04b-47c6-4307-bb21-75b068b5d078,False,False,True,False,0,0,...,1,0,0,False,1,2018-08-22,True,editorial_playlist,trackdone,fwdbtn
3,38_00012a3f-dff9-4560-b18b-103cfecf8a43,4,11,t_47cfab15-2b20-4dbb-91c1-db3b8cb70036,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,fwdbtn,trackdone
4,38_00012a3f-dff9-4560-b18b-103cfecf8a43,5,11,t_ab40acd6-2781-41e2-8c38-61cf49e02999,False,True,True,False,0,0,...,1,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,fwdbtn


In [16]:
test_group_predict

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
5,38_00012a3f-dff9-4560-b18b-103cfecf8a43,6,11,t_5478b5f2-f2bb-4ebb-808b-9c14ee50950c,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,fwdbtn,trackdone
6,38_00012a3f-dff9-4560-b18b-103cfecf8a43,7,11,t_8db3f89c-510b-4e30-87bc-887502bd9030,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
7,38_00012a3f-dff9-4560-b18b-103cfecf8a43,8,11,t_696ab8a4-321b-46fc-ac04-36fc07d37a30,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
8,38_00012a3f-dff9-4560-b18b-103cfecf8a43,9,11,t_ceb4e1b4-1cf6-432a-b199-107a0aef045b,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
9,38_00012a3f-dff9-4560-b18b-103cfecf8a43,10,11,t_b918f474-f39d-4bbc-89f9-463001ca1510,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
10,38_00012a3f-dff9-4560-b18b-103cfecf8a43,11,11,t_b076fb75-b1bd-44b3-a284-e7db55dddfb9,False,False,False,True,0,1,...,0,0,0,True,2,2018-08-22,True,editorial_playlist,trackdone,trackdone
